<div style="color:#007acc; text-align:center; font-size:32px; font-weight:bold; font-family:Arial, sans-serif;">
  Explore Data Analysis and Transform Purchase Log Data
</div>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plts
import seaborn as sns

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns',None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
df=pd.read_csv("../ingest_from_ITS/data/m952_purchase_2025-05-15_00-00-00_to_2025-06-15_23-59-59.csv") 
df

,@timestamp,agent.name,ecs.version,labels.app,labels.publisher,labels.source,related.ip,related.user,client.geo.continent_name,client.geo.region_iso_code,client.geo.city_name,client.geo.country_iso_code,client.geo.country_name,client.geo.region_name,client.geo.location.lon,client.geo.location.lat,client.ip,event.ingested,event.original,event.kind,event.module,event.action,event.its.game.id,event.its.payment.amount,event.its.payment.original_amount,event.its.payment.channel,event.its.payment.currency_unit,event.its.properties.national_id,event.its.properties.ref_id,event.its.properties.account_type,event.its.properties.response_data,event.its.properties.response_code,event.its.properties.signature,event.its.properties.discount_amount,event.its.properties.usd_exchange_rate,event.its.properties.partner_amount,event.its.properties.expired_time,event.its.properties.client_type,event.its.properties.character_id,event.its.properties.package_id,event.its.properties.error_message1,event.its.properties.discount_rate,event.its.properties.client_id,event.its.properties.error_message2,event.its.properties.wallet_currency_unit,event.its.properties.partner_id,event.its.properties.price,event.its.properties.product_id,event.its.properties.is_item,event.its.properties.exchange_rate,event.its.properties.channel_promotion_amount,event.its.properties.discount_code,event.its.properties.payment_amount,event.its.properties.result_code2,event.its.properties.server_id,event.its.properties.result_code1,event.its.properties.product_name,event.its.properties.discount_items,event.its.properties.transtime,event.its.properties.partner_trans_id,event.its.properties.fee_amount,event.its.properties.extra_info,event.its.properties.channel_rate,event.its.properties.is_complete1,event.its.properties.topup_amount,event.its.properties.channel_id,event.its.properties.is_complete2,event.its.properties.charge_amount,event.end,event.id,event.dataset,user.name,user.its.gender,user.id,event.its.properties.order_token,event.its.properties.service_email,event.its.properties.game_api_response,event.its.properties.channel_promotion_rate,user_agent.os.name
0,2025-05-14T17:00:22.352Z,CodaPayLog,8.9.0,m952,gamo,idgamo,['140.238.224.136'],"['2409010229286860', 'tancpqn46']",Asia,IN-MH,Mumbai,IN,India,Maharashtra,72.89,19.07,140.238.224.136,2025-05-14T17:00:40.618374703Z,"{""_id"":""6824cc375d6a6f6fc4b126dc"",""IsCompleted1"":1,""ResultCode1"":0,""ErrorMessage1"":""success"",""IsCompleted2"":1,""ResultCode2"":0,""ErrorMessage2"":"""",""ExpiredTime"":""2025-05-14T18:00:22.352Z"",""RequestTime"":""2025-05-14T17:00:22.352Z"",""ExecutedTime"":""2025-05-14T17:00:37.391Z"",""NationalID"":""VN"",""CurrencyUnit"":""USD"",""WalletCurrencyUnit"":""MIA"",""ExchangeRate"":25820,""PartnerID"":"""",""ClientID"":""idgamo"",""ClientType"":4,""TransactionID"":""2505150000213858"",""TransTime"":""2025-05-14T17:00:21.385Z"",""PartnerTransID"":""cb0edba853a04f19bf718135bc60f987"",""RechargeType"":1,""AccountType"":0,""CustomerID"":""2409010229286860"",""UserName"":""tancpqn46"",""CharacterID"":""133984"",""GameID"":""m952"",""ServerID"":""201"",""IsItem"":false,""PackageID"":""952003"",""ProductID"":""web.silkroad.pack.fund"",""ProductName"":""InvestmentFund"",""Price"":15,""ChannelID"":""payosqr"",""ChargeRate"":0.95,""ChargeAmount"":0.75,""ChannelPromotionRate"":0,""ChannelPromotionAmount"":0,""DiscountRate"":0,""DiscountAmount"":0,""FeeAmount"":0,""PaymentAmount"":15.75,""TopupAmount"":406665,""DiscountItems"":[],""PlatForm"":"""",""Signature"":""f9270f9fb19977412c558d30dba9ab93"",""GameApiReponse"":""{\r\n \""msg\"": \""Success\"",\r\n \""code\"": 1\r\n}"",""DiscountCode"":"""",""IsTest"":false,""IpAddress"":""140.238.224.136"",""ResponseData"":""{\""code\"":\""00\"",\""desc\"":\""success\"",\""success\"":true,\""data\"":{\""orderCode\"":2505150000213858,\""amount\"":406665,\""description\"":\""2505150000213858\"",\""accountNumber\"":\""4277227902720\"",\""reference\"":\""FT25135708573224\"",\""

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53319 entries, 0 to 53318
Data columns (total 79 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   @timestamp                                     53319 non-null  object 
 1   agent.name                                     53319 non-null  object 
 2   ecs.version                                    53319 non-null  object 
 3   labels.app                                     53319 non-null  object 
 4   labels.publisher                               53319 non-null  object 
 5   labels.source                                  53319 non-null  object 
 6   related.ip                                     42798 non-null  object 
 7   related.user                                   53319 non-null  object 
 8   client.geo.continent_name                      42798 non-null  object 
 9   client.geo.region_iso_code                     333

In [4]:
df.describe()

,client.geo.location.lon,client.geo.location.lat,event.its.payment.amount,event.its.payment.original_amount,event.its.properties.ref_id,event.its.properties.account_type,event.its.properties.discount_amount,event.its.properties.usd_exchange_rate,event.its.properties.partner_amount,event.its.properties.client_type,event.its.properties.character_id,event.its.properties.discount_rate,event.its.properties.error_message2,event.its.properties.price,event.its.properties.exchange_rate,event.its.properties.channel_promotion_amount,event.its.properties.discount_code,event.its.properties.payment_amount,event.its.properties.result_code2,event.its.properties.server_id,event.its.properties.result_code1,event.its.properties.fee_amount,event.its.properties.extra_info,event.its.properties.channel_rate,event.its.properties.is_complete1,event.its.properties.topup_amount,event.its.properties.is_complete2,event.its.properties.charge_amount,event.id,user.id,event.its.properties.order_token,event.its.properties.service_email,event.its.properties.game_api_response,event.its.properties.channel_promotion_rate
count,42798.00,42798.00,53319.00,53319.00,0.00,53319.00,53319.00,53319.00,53319.00,53319.00,53319.00,53319.00,0.00,53319.00,53319.00,53319.00,0.00,53319.00,53319.00,53319.00,53319.00,53319.00,15294.00,53319.00,53319.00,53319.00,53319.00,53319.00,53319.00,53319.00,0.00,0.00,0.00,8649.00
mean,72.10,23.53,10.42,10.42,NaN,0.45,0.00,1.00,124998.86,3.10,93847.78,0.00,NaN,10.09,25830.37,0.02,NaN,10.42,0.00,160.95,0.00,0.00,660824.27,0.68,1.00,154737.58,1.00,0.36,14195810412287988.00,2474602769585716.00,NaN,NaN,NaN,0.00
std,67.87,15.54,13.57,13.57,NaN,0.50,0.00,0.00,228619.85,1.35,127016.68,0.00,NaN,12.63,12.62,0.22,NaN,13.57,0.00,48.58,0.00,0.00,418787.89,0.43,0.00,257729.50,0.00,1.31,11268380525142032.00,82665658901631.25,NaN,NaN,NaN,0.00
min,-157.84,-53.15,-99.99,-99.99,NaN,0.00,0.00,1.00,0.00,1.00,3.00,0.00,NaN,0.75,25370.00,0.00,NaN,-99.99,0.00,21.00,0.00,0.00,39.00,0.00,1.00,-99.99,1.00,0.00,2505150000213858.00,6200000000063.00,NaN,NaN,NaN,0.00
25%,31.25,12.25,3.99,3.99,NaN,0.00,0.00,1.00,0.00,1.00,7342.00,0.00,NaN,3.99,25820.00,0.00,NaN,3.99,0.00,106.00,0.00,0.00,239050.25,0.00,1.00,9.99,1.00,0.00,2505311647240189.50,2410171729792849.00,NaN,NaN,NaN,0.00
50%,105.85,21.02,7.88,7.88,NaN,0.00,0.00,1.00,8.13,4.00,38972.00,0.00,NaN,7.50,25840.00,0.00,NaN,7.88,0.00,201.00,0.00,0.00,1054400.50,0.95,1.00,77520.00,1.00,0.08,25051609285057180.00,2503281905242140.00,NaN,NaN,NaN,0.00
75%,106.82,34.77,15.75,15.75,NaN,1.00,0.00,1.00,203619.20,4.00,135159.50,0.00,NaN,15.00,25840.00,0.00,NaN,15.75,0.00,203.00,0.00,0.00,1073673.75,0.95,1.00,203619.20,1.00,0.38,25060615237601680.00,2505201922380514.00,NaN,NaN,NaN,0.00
max,178.00,67.29,360.00,360.00,NaN,1.00,0.00,1.00,8139600.00,4.00,531015.00,0.00,NaN,300.00,25840.00,18.00,NaN,360.00,0.00,204.00,0.00,0.00,1100178.00,1.00,1.00,9000852.00,1.00,60.00,25061610213967540.00,2506152306178865.00,NaN,NaN,NaN,0.00


In [5]:
df.isnull().sum()

@timestamp                                         0
agent.name                                         0
ecs.version                                        0
labels.app                                         0
labels.publisher                                   0
                                               ...  
event.its.properties.order_token               53319
event.its.properties.service_email             53319
event.its.properties.game_api_response         53319
event.its.properties.channel_promotion_rate    44670
user_agent.os.name                             44670
Length: 79, dtype: int64

In [6]:
null_ratio = df.isnull().mean()
cols_to_drop = null_ratio[null_ratio > 0.9].index
print("column deleted:", list(cols_to_drop))
df = df.drop(columns=cols_to_drop)
df.shape

column deleted: ['event.its.properties.ref_id', 'event.its.properties.error_message2', 'event.its.properties.discount_code', 'event.its.properties.order_token', 'event.its.properties.service_email', 'event.its.properties.game_api_response']


(53319, 73)

In [7]:
df['@timestamp'] = pd.to_datetime(df['@timestamp'], errors='coerce', utc=True)
df.shape

(53319, 73)

In [8]:
len(df['user.id'].unique())

25625

In [9]:
df=df.rename(columns={'user.id':'vopenid'})
df['vopenid']=df['vopenid'].astype(str)

In [ ]:
data=df
# now = pd.Timestamp.now(tz='UTC')
now = pd.Timestamp("2025-06-16 12:00:00", tz="UTC")

In [11]:
def last_update_time(group):
    last_time = group['@timestamp'].max()
    return {
        'last_update_time': last_time
    }

In [12]:
def count_purchase(group):
    count = len(group)
    refund_count = (group['event.its.properties.payment_amount'] < 0).sum()
    count -= refund_count
    return {
        'num_purchase': max(count, 0)  
    }

In [13]:
def total_money_purchase(group):
    total=group['event.its.properties.payment_amount'].sum()
    if total < 0:
        total=0
    return {
        'total_purchase': total
    }

In [14]:
def count_purchase_per_day(group):
    if group.empty:
        return pd.Series({'count_purchase_per_day': 0})

    group['date'] = group['@timestamp'].dt.date
    daily_counts = group.groupby('date').size()

    avg = daily_counts.mean()
    
    return {
        'count_purchase_per_day': avg
    }

In [15]:
def std_amount_purchase(group):
    amounts = group['event.its.properties.payment_amount']
    amounts = amounts[amounts >= 0]
    
    if len(amounts) < 2:
        std = 0.0  
    else:
        std = amounts.std()
    
    return {
        'std_amount_purchase': std
    }

In [16]:
def max_money(group):
    max_mount=group['event.its.properties.payment_amount'].max()
    if max_mount < 0:
        max_mount=0
    return {
        'max_purchase':max_mount,
    }

In [17]:
def days_to_first_purchase(group):
    login_times = group[group['event.action'] == 'its_register']['@timestamp']
    purchase_times = group[group['event.action'] == 'its_purchase']['@timestamp']

    if login_times.empty or purchase_times.empty:
        return pd.Series({'days_to_first_purchase': pd.NA})

    first_login = login_times.min()
    first_purchase = purchase_times.min()
    
    return pd.Series({
        'days_to_first_purchase': (first_purchase - first_login).days
    })

In [18]:
def days_since_last_purchase(group):
    last_time = group['@timestamp'].max()
    return pd.Series({
        'days_since_last_purchase': (now - last_time).days
    })

In [19]:
def num_products(group):
    try:
        unique_products = group['event.its.properties.product_name'].nunique()
        return {
            'num_product_buyed': unique_products
        }
    except Exception as e:
        print(f"Error in num_products: {e}")
        return {
            'num_product_buyed': 0
        }

In [20]:
features_function=[
   last_update_time,
   count_purchase,
   total_money_purchase,
   count_purchase_per_day,
   std_amount_purchase,
   max_money,
   #days_to_first_purchase,
   days_since_last_purchase,
   num_products,
]

In [21]:
def extract_group_features(group):
    result={}
    for func in features_function:
        try:
            result.update(func(group))
        except Exception as e:
            print(f"Error at feature: {func}. Error: {e}")
    return pd.Series(result)

df_res = data.groupby('vopenid').apply(extract_group_features).reset_index()
df_res

C:\Users\GOSU.GOSU-TT-ZION1\AppData\Local\Temp\ipykernel_11644\3747869921.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_res = data.groupby('vopenid').apply(extract_group_features).reset_index()


,vopenid,last_update_time,num_purchase,total_purchase,count_purchase_per_day,std_amount_purchase,max_purchase,days_since_last_purchase,num_product_buyed
0,2206211830166192,2025-05-24 15:48:36.757000+00:00,1,0.79,1.00,0.00,0.79,22,1
1,2210221534402190,2025-05-26 05:14:28.541000+00:00,1,7.88,1.00,0.00,7.88,21,1
2,2210221828089318,2025-06-11 14:06:52.646000+00:00,2,3.98,2.00,0.00,1.99,4,2
3,2302080453545458,2025-05-26 03:22:07.961000+00:00,2,3.98,1.00,0.00,1.99,21,2
4,2302141307530184,2025-06-12 07:24:17.135000+00:00,1,1.99,1.00,0.00,1.99,4,1
...,...,...,...,...,...,...,...,...,...
25620,6210000928471,2025-05-16 11:35:30.848000+00:00,1,1.99,1.00,0.00,1.99,30,1
25621,6210000940123,2025-05-20 13:08:05.375000+00:00,1,7.88,1.00,0.00,7.88,26,1
25622,6210001017824,2025-06-12 11:28:47.307000+00:00,2,3.98,1.00,0.00,1.99,3,1
25623,6220001331202,2025-06-12 14:39:00.448000+00:00,1,1.99,1.00,0.00,1.99,3,1


In [22]:
df_res.to_csv("data/purchase_transform.csv",index=False,header=True)

In [23]:
df_res['std_amount_purchase'].min(),df_res['std_amount_purchase'].max()

(np.float64(0.0), np.float64(215.67463932970887))